In [1]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTMount
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

from lsst_efd_client import EfdClient

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.4.1/lib/python3.8/site-packages/jose/backends/cryptography_backend.py:23: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes, int_to_bytes


# Before the tests:

In [2]:
script = salobj.Controller("Script", index=1)

In [3]:
rot_sim = salobj.Remote(script.domain, "MTRotator")
await rot_sim.start_task

In [4]:
# this is to ensure that MTMount is connected and telemetry is send
rot_sim_connected = await rot_sim.evt_connected.aget(timeout=10.) # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
rot_sim_commandable = rot_sim-connected.command
rot_sim_telemetry_working = rot_sim_connected.telemetry

if not rot_sim_connected or not rot_sim_telemetry_working:
    raise Exception("Mount not connected or telemetry not being received.")

TimeoutError: 

In [ ]:
#This is to check what the controller state is:
controller_state = await rot_sim.evt_controllerState.aget()
state = controller_state.controllerState
#offline_substate = controller_state.offlineSubstate
print(state)

In [ ]:
#This is to check MTMount is commandable through SAL:
commandable_by_dds = await rot_sim.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")

In [ ]:
#Print the available MTMount events
print(rot_sim.salinfo.event_names)

In [ ]:
#Check if MTMount telemetry is getting into the EFD
#Add telemetry as needed
#For the "setCompensateMode" of the Hexapods the "angle" of the rotator is needed
rot_sim_Elevation= await rot_sim.tel_elevation.aget(timeout=10.)
print("Simulated rotator angle", rot_sim_Elevation.angleActual)

In [ ]:
#Send the MTMount simulator component into STANDBY
await salobj.set_summary_state(rot_sim, salobj.State.STANDBY)

In [ ]:
#Check if the input to the EFD has stopped
rot_sim_Elevation= await rot_sim.tel_elevation.aget(timeout=10.)
print("Simulated rotator angle", rot_sim_Elevation.angleActual)

In [ ]:
#Stop the script controller
await script.close()

In [ ]:
#Start the MTMount controller
rot_control = salobj.Controller(name="Rotator")

In [ ]:
#Check if the input to the EFD has started
rot_control_Elevation= await rot_control.tel_elevation.aget(timeout=10.)
print("Rotator controller angle", rot_control_Elevation.angleActual)

# Set the MTMount Controller to the needed values and exectute the tests from another notebook:

In [ ]:
# Set the elevation angle. Do this for every desired elevation angle.
rotator.evt_target.set_put(position=0)

# After the tests:

In [ ]:
#Stop the MTMount controller
await rot_control.close()

In [ ]:
#Check that no more telemetry from the MTMount controller is going into the EFD
rot_control_Elevation= await rot_contorl.tel_elevation.aget(timeout=10.)
print("Mount controller elevation", rot_control_Elevation.angleActual)

In [ ]:
#Start the script controller
script = salobj.Controller("Script", index=1)

In [ ]:
#Start the mount Simluator controller
await rot_sim.start_task

In [ ]:
#Send the MTMount simulator component into ENABLE again
await salobj.set_summary_state(rot, salobj.State.ENABLE)

In [ ]:
#Check that the EFD is getting MTMount siumlator telemetry

In [ ]:
#print available commands
print(rot.salinfo.command_names)